# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [11]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
# pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfFile = open("Data/UIdaho2019/small-grains-report_2019.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 46

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  producing well in irrigated southern Idaho conditions.
1 :  In southern Idaho, Lenetah has average yield, test weight and plump, slightly earlier than average heading date and is two inches taller in height.
2 :  Lenetah yields and lodging are similar Champion.
3 :  Millennium (UT004603)  a six-row spring feed barley released in 2000 through Utah AES, Millennium does very well under irrigation, and has been in the top-yielding groups under dryland conditions when moisture was adequate.
4 :  Millennium also has excellent straw strength, showing minimal lodging even under high-yield conditions.
5 :  Millennium is of average height and protein for six-rowed varieties, and taller than two-rowed lines.
6 :  Millennium has lower test weight and plump, and heads several days earlier than average.
7 :  Millennium is susceptible to FHB and will accumulate high levels of DON.
8 :  Oreana  (BZ509-448)  a two-rowed feed barley originally developed through WestBred, Oreana is carried by Highla

## Read in Per-line named entity file and match entities to sentence positions.

In [13]:
import re
import csv
import pandas as pd

#fname = "Data/DavisLJ11/barley_p"+str(pageNumber)+"_ner.txt"
fname = "Data/UIdaho2019/barley_p"+str(pageNumber-40)+"_ner.txt" # Note p1 = 41 in the document

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


    Index                                           Sentence        Phrase  \
0       1  In southern Idaho, Lenetah has average yield, ...         Idaho   
1       1  In southern Idaho, Lenetah has average yield, ...       Lenetah   
2       1  In southern Idaho, Lenetah has average yield, ...         yield   
3       1  In southern Idaho, Lenetah has average yield, ...   test weight   
4       1  In southern Idaho, Lenetah has average yield, ...         plump   
..    ...                                                ...           ...   
134    35  Released in 2008, Endeavor has improved malt q...          2008   
135    35  Released in 2008, Endeavor has improved malt q...      Endeavor   
136    35  Released in 2008, Endeavor has improved malt q...  malt quality   
137    35  Released in 2008, Endeavor has improved malt q...         yield   
138    35  Released in 2008, Endeavor has improved malt q...       Charles   

            MatchInfo  
0     (12, 17, 'GPE')  
1    (19, 26, '

## Create a function to clean up overlapping intervals

In [14]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [15]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
print("TRAIN_DATA = [")
maxr = len(records)
for i in range(0,maxr):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]})", end='')
    if (i == maxr-1):
        print()
    else:
        print(",")

print("]")

TRAIN_DATA = [
    ('In southern Idaho, Lenetah has average yield, test weight and plump, slightly earlier than average heading date and is two inches taller in height.', {'entities': [(12, 17, 'GPE'), (19, 26, 'CVAR'), (39, 44, 'TRAT'), (46, 57, 'TRAT'), (62, 67, 'TRAT'), (99, 111, 'TRAT'), (140, 146, 'TRAT')]}),
    ('Oreana showed intermediate reaction to FHB and showed higher levels of DON accumulated in the seed.', {'entities': [(0, 6, 'CVAR'), (14, 35, 'PPTD'), (39, 42, 'PATH'), (61, 98, 'TRAT')]}),
    ('Xena (BZ594-19)  a two-rowed spring feed barley released by Western Plant Breeders that is now handled by Highland Specialty Grain.', {'entities': [(0, 4, 'CVAR'), (6, 14, 'ALAS'), (19, 28, 'TRAT'), (29, 35, 'TRAT'), (36, 40, 'TRAT'), (41, 47, 'CROP'), (60, 82, 'ORG'), (106, 130, 'ORG')]}),
    ('Xena has had very high yields over the locations tested from 2010-2019, similar to Champion.', {'entities': [(0, 4, 'CVAR'), (23, 29, 'TRAT'), (61, 65, 'DATE'), (66, 70, 'DATE'), (83, 9

In my current process, I am writing the above content to a file e.g., `Data/DavisLJ11/barley_p5_td.py`, adding any manual corrections (usually PED and JRNL entries) and then running the script `python3 py2json.py --doc 'BarCvDescLJ11.pdf' --url 'https://smallgrains.ucdavis.edu/cereal_files/BarCvDescLJ11.pdf' --chunk 5 Data/DavisLJ11/barley_p5_td.py Data/DavisLJ11/barley_p5_td.json` to create the JSON file for Training.